In [24]:
import pandas as pd

df = pd.read_csv('df_active.csv')

# Find the indices of the rows where taskState contains 'setNavigationTarget'
index_set_navigation_target = df[df['taskState'].str.contains('setNavigationTarget', na=False)].index

# Loop through each identified index and replace the values in 'mapInteractions' for the following three rows
for idx in index_set_navigation_target:
    df.loc[idx + 1:idx + 3, 'mapInteractions'] = ''

df.to_csv('see.csv')


In [18]:

# Create a new DataFrame with 'active_event' and the 'if_pan' and 'if_zoom' columns
new_df = df[['active_event']].copy()

# Add the 'if_pan' column and set its value based on the condition
new_df['if_pan'] = df['mapInteractions'].apply(lambda x: 1 if 'mapLog:positionXYZ' in str(x) else 0)
new_df['if_zoom'] = df['mapInteractions'].apply(lambda x: 1 if 'mapLog:zoom' in str(x) else 0)
new_df['if_selflocation'] = df['mapInteractions'].apply(lambda x: 1 if 'FocusOnPlayer' in str(x) else 0)
new_df['if_follow'] = df['mapInteractions'].apply(lambda x: 1 if 'buttonOnFollowPlayer' in str(x) else 0)

# Group by 'active_event' and use max to set if_pan and if_zoom to 1 if any row in the group has if_pan or if_zoom as 1
grouped_df = new_df.groupby('active_event').agg({'if_pan': 'max', 'if_zoom': 'max','if_selflocation': 'max', 'if_follow': 'max' }).reset_index()

print(grouped_df)

      active_event  if_pan  if_zoom  if_selflocation  if_follow
0                1       1        0                0          0
1                2       1        0                0          0
2                3       1        0                0          0
3                4       1        1                0          0
4                5       1        0                0          0
...            ...     ...      ...              ...        ...
3118          3119       1        0                0          0
3119          3120       1        0                0          0
3120          3121       1        0                0          0
3121          3122       1        0                0          0
3122          3123       1        0                0          0

[3123 rows x 5 columns]


In [23]:
result = pd.read_csv('result_indi_env.csv')

result_indi_env_interaction = result.merge(grouped_df, how= 'left', left_on= 'event_number', right_on='active_event')
result_indi_env_interaction = result_indi_env_interaction.drop(columns= 'active_event')
result_indi_env_interaction = result_indi_env_interaction.drop(columns= 'Unnamed: 0')


result_indi_env_interaction.to_csv('result_indi_env_interaction.csv')

In [25]:
# Define the formula for the linear mixed model
import statsmodels.api as sm
import statsmodels.formula.api as smf
df = pd.read_csv('result_indi_env_interaction.csv')
formula = 'total_inactive_time ~ traffic_density + sbsod_score + age + mobile_map_freq_num + ifcross_preactive + ifcross_inactive + completion_rate + if_follow'

# Fit the model with random intercepts for 'participant'
model = smf.mixedlm(formula, df, groups=df['participant'])

# Fit the model and print the summary
result = model.fit()
print(result.summary())

              Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: total_inactive_time
No. Observations: 3117    Method:             REML               
No. Groups:       54      Scale:              159.0670           
Min. group size:  21      Log-Likelihood:     -12389.3962        
Max. group size:  169     Converged:          Yes                
Mean group size:  57.7                                           
-----------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept               4.557    6.884  0.662 0.508 -8.935 18.050
traffic_density         1.414    0.461  3.071 0.002  0.512  2.317
sbsod_score             2.460    0.956  2.573 0.010  0.587  4.334
age                     2.560    1.845  1.388 0.165 -1.056  6.176
mobile_map_freq_num    -1.829    1.759 -1.039 0.299 -5.277  1.620
ifcross_preactive       

In [26]:
# Fit a GLM with Gaussian family
model = smf.glm('total_inactive_time ~ traffic_density + sbsod_score + age + mobile_map_freq_num + ifcross_preactive + ifcross_inactive + completion_rate + if_follow', 
                 data=df, family=sm.families.Gaussian()).fit()

print(model.summary())


                  Generalized Linear Model Regression Results                  
Dep. Variable:     total_inactive_time   No. Observations:                 3117
Model:                             GLM   Df Residuals:                     3108
Model Family:                 Gaussian   Df Model:                            8
Link Function:                identity   Scale:                          191.90
Method:                           IRLS   Log-Likelihood:                -12611.
Date:                 Mon, 08 Jul 2024   Deviance:                   5.9644e+05
Time:                         22:40:25   Pearson chi2:                 5.96e+05
No. Iterations:                      3   Pseudo R-squ. (CS):             0.2762
Covariance Type:             nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               